In [46]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = None

RAM8 = pd.read_csv('RAM-8_raw_data.csv')
RAM9 = pd.read_csv('RAM-9_raw_data.csv')

RAM9['Stain'] = RAM9['Stain'].str.replace('GFP', 'EGFP')
RAM9['Stain'] = RAM9['Stain'].str.replace('w/', 'with')

# standardizing stain type names
stains = []
for stain in list(RAM9.Stain):
    if stain == 'c-Fos':
        stain = 'single_c-Fos'
        
    if stain == 'mKate2':
        stain = 'single_mKate2'
        
    if stain == 'EGFP':
        stain = 'single_EGFP'
        
    if 'Triple-labeled' in stain:
        stain = stain.replace('Triple-labeled', 'triple')
        
    stains.append(stain)
    
RAM9['Stain'] = stains

print(f'Stain type(s) in RAM-9 but NOT RAM-8: {set(np.unique(RAM9.Stain)) - set(np.unique(RAM8.Stain))}')

# standardizing rat_n
RAM9['Rat_n'] = RAM9.Rat_n.str.replace('_', '')
RAM9['Rat_n'] = RAM9.Rat_n.str.replace('rat', 'RAM-9_rat')

RAM8['Rat_n'] = RAM8.Rat_n.str.replace('rat', 'RAM-8_rat')

# standardizing columns (prep for concat)
RAM9 = RAM9.drop(columns=['Treatment'])

RAM8_col = set(RAM8.columns)
RAM9_col = set(RAM9.columns)

print(f'n of RAM-8 columns: {len(RAM8_col)}')
print(f'n of RAM-9 columns: {len(RAM9_col)}')

print(f'Symmetric difference of RAM-8 and RAM-9 columns: {RAM9_col.symmetric_difference(RAM8_col)}')
print(f'RAM-8 columns are equal to RAM-9 columns: {RAM9_col == RAM8_col}')

Stain type(s) in RAM-9 but NOT RAM-8: {'EGFP_coloc_with_mKate2'}
n of RAM-8 columns: 31
n of RAM-9 columns: 31
Symmetric difference of RAM-8 and RAM-9 columns: set()
RAM-8 columns are equal to RAM-9 columns: True


C:\Users\jonat\AppData\Local\Temp\ipykernel_15200\4114633555.py:7: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  RAM9 = pd.read_csv('RAM-9_raw_data.csv')


In [47]:
# concat! and dropping more useless columns
RAM89 = pd.concat([RAM8, RAM9])
RAM89 = RAM89.drop(columns=['Unnamed: 0','Ind_var_group', 'Indi_var_group'])
RAM89 = RAM89.dropna()


def convert_coord_tuple(df):
    coords = []
    for coord in df.Coord:
        coords.append(tuple(float(n) for n in coord.replace('(', '').replace(')','').replace(' ','').split(',')))

    return coords

RAM89['Coord'] = convert_coord_tuple(RAM89)
RAM89

,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,Integrated_Density,Max,Mean,Mean-Background,Min,Round,Solidity,Stain,StdDev,Subject,XM,YM,directory,roiName,Rat_n,Coord,Sex,Dox,React
0,1,1.092,_5_3_2021_at_8-34,43.489,56.949,1.0,0.873,RAM008,3.4_PL_A_2,3663.566,217.394,141.189,84.241,59.153999999999996,0.811,0.838,single_c-Fos,43.536,10.0,345.615,79.261,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00002,RAM-8_rat10C,"(345.615, 79.26100000000001)",F,DOX_ON,FR1
1,2,1.000,_5_3_2021_at_8-34,64.533,56.949,2.0,0.968,RAM008,3.4_PL_A_2,5381.915,270.618,140.347,83.398,59.455,0.948,0.911,single_c-Fos,54.939,10.0,57.237,77.429,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00010,RAM-8_rat10C,"(57.236999999999995, 77.429)",F,DOX_ON,FR1
2,3,1.169,_5_3_2021_at_8-34,66.637,56.949,3.0,0.996,RAM008,3.4_PL_A_2,13203.354,478.177,255.088,198.139,78.491,0.846,0.969,single_c-Fos,89.950,10.0,13.054,117.141,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00007,RAM-8_rat10C,"(13.054, 117.141)",F,DOX_ON,FR1
3,4,1.302,_5_3_2021_at_8-34,25.252,56.949,4.0,0.562,RAM008,3.4_PL_A_2,886.135,169.740,92.041,35.092,57.24,0.432,0.545,single_c-Fos,26.349,10.0,368.777,231.472,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00003,RAM-8_rat10C,"(368.777, 231.472)",F,DOX_ON,FR1
4,5,1.083,_5_3_2021_at_8-34,59.623,56.949,5.0,0.981,RAM008,3.4_PL_A_2,8228.827,374.593,194.964,138.015,62.191,0.913,0.924,single_c-Fos,85.767,10.0,107.768,29.112,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00009,RAM-8_rat10C,"(107.76799999999999, 29.112)",F,DOX_ON,FR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18180,4,1.171,_5_18_2022_at_16-56,48.399,5.675,4.0,0.447,RAM-9,RAM-9_7B_D_3,329.735,32.027,12.488,6.813,6.027,0.390,0.457,single_mKate2,6.034,7D,260.259,207.464,data/RAM-9_7B_D_3/RAM-9_7B_D_3_{single}_{DAPI}...,0247c-0310,RAM-9_rat7B,"(260.259, 207.46400000000003)",F,DOX_ON,FR1
18181,5,1.000,_5_18_2022_at_16-56,46.997,5.675,5.0,0.333,RAM-9,RAM-9_7B_D_3,312.926,30.848,12.334,6.658,5.805,0.322,0.328,single_mKate2,6.451,7D,193.557,195.296,data/RAM-9_7B_D_3/RAM-9_7B_D_3_{single}_{DAPI}...,0232c-0231,RAM-9_rat7B,"(193.55700000000002, 195.296)",F,DOX_ON,FR1
18182,1,1.188,_5_18_2022_at_16-56,46.295,5.675,1.0,0.836,RAM-9,RAM-9_7B_D_3.tif,1496.300,91.041,37.996,32.321,6.04,0.703,0.815,mKate2_coloc_with_EGFP,21.339,7D,347.029,84.184,data/double-label_results/RAM-9_7B_D_3_{double...,0099c-0413,RAM-9_rat7B,"(347.029, 84.184)",F,DOX_ON,FR1
18183,2,1.171,_5_18_2022_at_16-56,48.399,5.675,2.0,0.447,RAM-9,RAM-9_7B_D_3.tif,329.735,32.027,12.488,6.813,6.027,0.390,0.457,mKate2_coloc_with_EGFP,6.034,7D,260.259,207.464,data/double-label_results/RAM-9_7B_D_3_{double...,0247c-0310,RAM-9_rat7B,"(260.259, 207.46400000000003)",F,DOX_ON,FR1


In [48]:
# ok I realized that I'll have to normalize each group to its own norm condition first to calcualte
# normalized intensity then we can concat

RAM8 = RAM89[RAM89.Rat_n.str.contains('RAM-8')]
RAM9 = RAM89[RAM89.Rat_n.str.contains('RAM-9')]

RAM8_bystain = [RAM8[RAM8['Stain'] == stain] for stain in np.unique(RAM8.Stain)]
RAM9_bystain = [RAM9[RAM9['Stain'] == stain] for stain in np.unique(RAM9.Stain)]


def drop_dups_norm(lst_df_by_stain, normcondition1, normcondition2):
    
    ### dropping duplicates ###
    lst_df_dropped_duplicates = []

    for df in lst_df_by_stain:
        lst_df_dropped_duplicates.append(
            df.drop_duplicates(subset=['Area', 'Background', 'CellNumber', 'Circ.',
           'Experiment', 'FileName', 'Integrated_Density', 'Max', 'Mean','Min', 'Round',
           'Solidity', 'Stain', 'StdDev', 'Subject', 'XM', 'YM', 'roiName', 'Coord'])
        )   
            ##### PLEASE CHECK THESE COLUMN NAMES ##### 
            # (should be fine for most up to date pipsqueak versions)

    ### Normalize ###
    norm_means = []
    lst_df_by_stain_dropped_duplicates_NORM = []

    for df in lst_df_dropped_duplicates:
        # grabbing the mean of our norm condition
        df_norm = df[(df['Dox'] == normcondition1) & (df['React'] == normcondition2)]  ### norm condition
        norm_mean = df_norm['Mean-Background'].mean()
        norm_means.append(norm_mean)

        # dividing our Mean-Background by Veh mean
        df_copy = df
        df_copy['Mean-Background_NORM'] = df['Mean-Background'] / norm_mean

        # storing new DataFrame in a list
        lst_df_by_stain_dropped_duplicates_NORM.append(df_copy)
    
    # check that the mean of the normalized data is = to 1 (or something very close)
    for df in lst_df_by_stain_dropped_duplicates_NORM:
        a = df[(df['Dox'] == normcondition1) & (df['React'] == normcondition2)]
        print(np.unique(a.Stain).item(), np.average(a['Mean-Background_NORM']))

    ### export csvs ###
    #file_names_norm = [name.replace('.csv','_NORM.csv') for name in csv_names]

    #for i in range(len(file_names_norm)):
    #    lst_df_by_stain_dropped_duplicates_NORM[i].to_csv(file_names_norm[i])
        
    return lst_df_by_stain_dropped_duplicates_NORM

RAM8_bystain_NORM = drop_dups_norm(RAM8_bystain, 'DOX_ON', 'FR1')
RAM9_bystain_NORM = drop_dups_norm(RAM9_bystain, 'DOX_ON', 'FR1')

EGFP_coloc_with_c-Fos 1.0000000000000004
c-Fos_coloc_with_EGFP 0.9999999999999994
c-Fos_coloc_with_mKate2 1.0
mKate2_coloc_with_EGFP 1.0
mKate2_coloc_with_c-Fos 1.0
single_EGFP 0.9999999999999983
single_c-Fos 1.0000000000000004
single_mKate2 1.0
triple_EGFP 1.0
triple_c-Fos 1.0
triple_mKate2 1.0
EGFP_coloc_with_c-Fos 0.9999999999999998
EGFP_coloc_with_mKate2 0.9999999999999999
c-Fos_coloc_with_EGFP 1.0
c-Fos_coloc_with_mKate2 1.0
mKate2_coloc_with_EGFP 1.0000000000000002
mKate2_coloc_with_c-Fos 1.0
single_EGFP 1.0000000000000022
single_c-Fos 0.9999999999999998
single_mKate2 1.0000000000000002
triple_EGFP 1.0
triple_c-Fos 1.0
triple_mKate2 0.9999999999999999


In [49]:
RAM8_cfos_median = np.median(RAM8_bystain_NORM[6]['Mean-Background_NORM'])
RAM9_cfos_median = np.median(RAM9_bystain_NORM[7]['Mean-Background_NORM'])

RAM8_cfos = RAM8_bystain_NORM[6]
RAM9_cfos = RAM9_bystain_NORM[7]

RAM8_cfosonmkate = RAM8_bystain_NORM[2]
RAM9_cfosonmkate = RAM9_bystain_NORM[3]

RAM8_mkateoncfos = RAM8_bystain_NORM[4]
RAM9_mkateoncfos = RAM9_bystain_NORM[5]

print(RAM8_cfos_median)
print(RAM9_cfos_median)

0.7820307760391789
0.6981625336482719


In [50]:
# ok let's just focus on RAM 8 for now then repeat whatever we do for RAM 9

def new_filenames(df):
    filenames = []
    for rat_n, filename in zip(list(df['Rat_n']), list(df['FileName'])):
        f = '_'.join([rat_n, filename])
        f = '_'.join(f.split('_')[:-1])
        
        filenames.append(f)
        
    df['FileName'] = filenames
    
    return df

RAM8_cfos = new_filenames(RAM8_cfos)
RAM8_cfosonmkate = new_filenames(RAM8_cfosonmkate)
RAM8_mkateoncfos = new_filenames(RAM8_mkateoncfos)

In [51]:
# check that we have the same image names across stain types
print(set(RAM8_cfos.FileName).issuperset(set(RAM8_mkateoncfos.FileName)))

def distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

images = np.unique(RAM8_cfosonmkate.FileName)


mkateoncfos_nearest_cfos = []
for image in images:
    cfos = RAM8_cfos[RAM8_cfos.FileName == image].copy()
    mkateoncfos = RAM8_mkateoncfos[RAM8_mkateoncfos.FileName == image].copy()
    
    cfos_ints = []
    nearest_cfos_coord = []
    for coord1 in mkateoncfos.Coord:
        coords_d = [(coord2, distance(coord1, coord2)) for coord2 in list(cfos.Coord)]
        min_coord, min_d = min(coords_d, key=lambda a:a[1])
        
        cfos_int = np.array(cfos[cfos.Coord == min_coord]['Mean-Background_NORM']).item()
        cfos_ints.append(cfos_int)
        
        nearest_cfos_coord.append(min_coord)
        
    mkateoncfos['Nearest_cfos_coord'] = nearest_cfos_coord
    mkateoncfos['Nearest_cfos_int'] = cfos_ints
    
    mkateoncfos_nearest_cfos.append(mkateoncfos)
    
RAM8_mkateoncfos_nearestcfos = pd.concat(mkateoncfos_nearest_cfos)
    

True


In [52]:
RAM9_cfos = new_filenames(RAM9_cfos)
RAM9_cfosonmkate = new_filenames(RAM9_cfosonmkate)
RAM9_mkateoncfos = new_filenames(RAM9_mkateoncfos)

# check that we have the same image names across stain types
print(set(RAM9_cfos.FileName).issuperset(set(RAM9_mkateoncfos.FileName)))


images = np.unique(RAM9_cfosonmkate.FileName)


mkateoncfos_nearest_cfos = []
for image in images:
    cfos = RAM9_cfos[RAM9_cfos.FileName == image].copy()
    mkateoncfos = RAM9_mkateoncfos[RAM9_mkateoncfos.FileName == image].copy()
    
    cfos_ints = []
    nearest_cfos_coord = []
    for coord1 in mkateoncfos.Coord:
        coords_d = [(coord2, distance(coord1, coord2)) for coord2 in list(cfos.Coord)]
        min_coord, min_d = min(coords_d, key=lambda a:a[1])
        
        cfos_int = np.array(cfos[cfos.Coord == min_coord]['Mean-Background_NORM'])[0]
        cfos_ints.append(cfos_int)
        
        nearest_cfos_coord.append(min_coord)
        
    mkateoncfos['Nearest_cfos_coord'] = nearest_cfos_coord
    mkateoncfos['Nearest_cfos_int'] = cfos_ints
    
    mkateoncfos_nearest_cfos.append(mkateoncfos)
    
RAM9_mkateoncfos_nearestcfos = pd.concat(mkateoncfos_nearest_cfos)

True


In [53]:
RAM8_nearest_cfos = RAM8_mkateoncfos_nearestcfos['Nearest_cfos_int']
RAM9_nearest_cfos = RAM9_mkateoncfos_nearestcfos['Nearest_cfos_int']

cfos_highlow = []
for cfos_int in RAM8_nearest_cfos:
    if cfos_int < RAM8_cfos_median:
        cfos_highlow.append('c-Fos low')
    elif cfos_int >= RAM8_cfos_median:
        cfos_highlow.append('c-Fos high')
RAM8_mkateoncfos_nearestcfos['c-Fos Intensity Bin'] = cfos_highlow

cfos_highlow = []
for cfos_int in RAM9_nearest_cfos:
    if cfos_int < RAM9_cfos_median:
        cfos_highlow.append('c-Fos low')
    elif cfos_int >= RAM9_cfos_median:
        cfos_highlow.append('c-Fos high')
RAM9_mkateoncfos_nearestcfos['c-Fos Intensity Bin'] = cfos_highlow

RAM89_mkatedouble_cfoshighlow = pd.concat([RAM8_mkateoncfos_nearestcfos, RAM9_mkateoncfos_nearestcfos])

In [54]:
df = RAM89_mkatedouble_cfoshighlow 

cfos_high_by_image = []
cfos_low_by_image = []
images = []
for rat in np.unique(df['Rat_n']):
    df_rat = df[df.Rat_n == rat]
    
    for image in np.unique(df_rat['FileName']):
        df_rat_image = df_rat[df_rat.FileName == image]
        cfos_high_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos high']))
        cfos_low_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos low']))
        images.append(image)

RAM89_mkatedouble_cfoshighlow_counts = pd.DataFrame()
RAM89_mkatedouble_cfoshighlow_counts['Image'] = images
RAM89_mkatedouble_cfoshighlow_counts['mKate2 on c-Fos counts, c-Fos high'] = cfos_high_by_image
RAM89_mkatedouble_cfoshighlow_counts['mKate2 on c-Fos counts, c-Fos low'] = cfos_low_by_image

In [55]:
rat_ns = []
for image in RAM89_mkatedouble_cfoshighlow_counts['Image']:
    rat_n = '_'.join(image.split('_')[0:2])
    rat_ns.append(rat_n)
    
RAM89_mkatedouble_cfoshighlow_counts['Rat_n'] = rat_ns

rats = np.unique(RAM89.Rat_n)

SUM_mkatedouble_cfos_high = []
SUM_mkatedouble_cfos_low = []
rat_ns = []
for rat in rats:
    
    if rat in set(RAM89_mkatedouble_cfoshighlow_counts['Rat_n']):
        df = RAM89_mkatedouble_cfoshighlow_counts[RAM89_mkatedouble_cfoshighlow_counts['Rat_n'] == rat]
        SUM_mkatedouble_cfos_high.append(np.sum(df['mKate2 on c-Fos counts, c-Fos high']))
        SUM_mkatedouble_cfos_low.append(np.sum(df['mKate2 on c-Fos counts, c-Fos low']))
        rat_ns.append(rat)
        
    elif rat not in set(RAM89_mkatedouble_cfoshighlow_counts['Rat_n']):
        SUM_mkatedouble_cfos_high.append(0)
        SUM_mkatedouble_cfos_low.append(0)
        rat_ns.append(rat)
        
    
RAM89_mkatedouble_by_cfos = pd.DataFrame()
RAM89_mkatedouble_by_cfos['Rat_n'] = rat_ns
RAM89_mkatedouble_by_cfos['Stain'] = 'mKate2 coloc with c-Fos'
RAM89_mkatedouble_by_cfos['Sum Raw Cell n, c-Fos high'] = SUM_mkatedouble_cfos_high
RAM89_mkatedouble_by_cfos['Sum Raw Cell n, c-Fos low'] = SUM_mkatedouble_cfos_low

In [56]:
rats = np.unique(RAM89.Rat_n)
sex = []
dox = []
react = []
for rat in rats:
    sex.append(np.unique(RAM89[RAM89.Rat_n == rat]['Sex']).item())
    dox.append(np.unique(RAM89[RAM89.Rat_n == rat]['Dox']).item())
    react.append(np.unique(RAM89[RAM89.Rat_n == rat]['React']).item())
    
d_sex = dict(zip(rats, sex))
d_dox = dict(zip(rats, dox))
d_react = dict(zip(rats, react))

d_imgn = {
    'RAM-8_rat10C' : 3,
    'RAM-8_rat12C' : 5, 
    'RAM-8_rat13C' : 5, 
    'RAM-8_rat15C' : 6,
    'RAM-8_rat16C' : 4, 
    'RAM-8_rat1C' : 6, 
    'RAM-8_rat3C' : 3, 
    'RAM-8_rat4C' : 3,
    'RAM-8_rat5C' : 5, 
    'RAM-8_rat7C' : 1, 
    'RAM-8_rat8C' : 3, 
    'RAM-9_rat11B' : 3,
    'RAM-9_rat12B' : 4, 
    'RAM-9_rat14B' : 4, 
    'RAM-9_rat1B' : 4, 
    'RAM-9_rat2B' : 4,
    'RAM-9_rat3B' : 4, 
    'RAM-9_rat5B' : 4, 
    'RAM-9_rat7B' : 4
    
}

rats = []
sex = []
dox = []
react = []
img_n = []
mean_cell_n_cfoshigh = []
mean_cell_n_cfoslow = []
for rat in RAM89_mkatedouble_by_cfos.Rat_n:
    sex.append(d_sex[rat])
    dox.append(d_dox[rat])
    react.append(d_react[rat])
    img_n.append(d_imgn[rat])    
    sum_n = np.array(RAM89_mkatedouble_by_cfos[RAM89_mkatedouble_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos high']).item()
    mean_cell_n_cfoshigh.append(sum_n / d_imgn[rat])
    sum_n = np.array(RAM89_mkatedouble_by_cfos[RAM89_mkatedouble_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos low']).item()
    mean_cell_n_cfoslow.append(sum_n / d_imgn[rat])

    rats.append(rat)
    

In [57]:
RAM89_mkatedouble_by_cfos = RAM89_mkatedouble_by_cfos.drop(columns=['Rat_n'])

RAM89_mkatedouble_by_cfos['Total Image n'] = img_n
RAM89_mkatedouble_by_cfos['Mean Cell n, c-Fos High'] = mean_cell_n_cfoshigh
RAM89_mkatedouble_by_cfos['Mean Cell n, c-Fos Low'] = mean_cell_n_cfoslow
RAM89_mkatedouble_by_cfos['Rat_n'] = rats
RAM89_mkatedouble_by_cfos['Dox'] = dox
RAM89_mkatedouble_by_cfos['Sex'] = sex
RAM89_mkatedouble_by_cfos['React'] = react

In [58]:
RAM89_mkatedouble_by_cfos = RAM89_mkatedouble_by_cfos.sort_values(by=['Dox', 'React','Rat_n', 'Sex'])

In [59]:
RAM89_mkatedouble_by_cfos

,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low",Total Image n,"Mean Cell n, c-Fos High","Mean Cell n, c-Fos Low",Rat_n,Dox,Sex,React
3,mKate2 coloc with c-Fos,19,9,6,3.166667,1.500000,RAM-8_rat15C,DOX_OFF,F,FR1
7,mKate2 coloc with c-Fos,14,32,3,4.666667,10.666667,RAM-8_rat4C,DOX_OFF,M,FR1
9,mKate2 coloc with c-Fos,4,0,1,4.000000,0.000000,RAM-8_rat7C,DOX_OFF,M,FR1
10,mKate2 coloc with c-Fos,22,20,3,7.333333,6.666667,RAM-8_rat8C,DOX_OFF,M,FR1
11,mKate2 coloc with c-Fos,6,22,3,2.000000,7.333333,RAM-9_rat11B,DOX_OFF,F,FR1
16,mKate2 coloc with c-Fos,48,35,4,12.000000,8.750000,RAM-9_rat3B,DOX_OFF,M,FR1
5,mKate2 coloc with c-Fos,6,3,6,1.000000,0.500000,RAM-8_rat1C,DOX_OFF,M,NR
1,mKate2 coloc with c-Fos,0,0,5,0.000000,0.000000,RAM-8_rat12C,DOX_OFF,F,VR5
2,mKate2 coloc with c-Fos,33,8,5,6.600000,1.600000,RAM-8_rat13C,DOX_OFF,F,VR5
6,mKate2 coloc with c-Fos,81,46,3,27.000000,15.333333,RAM-8_rat3C,DOX_OFF,M,VR5


In [60]:
set(RAM89.Rat_n) - set(RAM89_mkatedouble_by_cfos.Rat_n) 

set()

In [61]:
RAM89_mkatedouble_by_cfos.to_csv('RAM-8,9 mKate2 coloc with c-Fos Mean Cell ns, c-Fos Median Split.csv')

In [62]:
RAM8_mkateoncfos_nearestcfos.to_csv('RAM-8,9, mKate2 coloc with c-Fos, c-Fos Median Split RAW.csv')

In [63]:
# ok now let's count how many c-fos high/low we have across all c-fos cells in general

# starting with where we defined the medians and c-fos dataframes
RAM8_cfos_median = np.median(RAM8_bystain_NORM[6]['Mean-Background_NORM'])
RAM9_cfos_median = np.median(RAM9_bystain_NORM[7]['Mean-Background_NORM'])

RAM8_cfos = RAM8_bystain_NORM[6]
RAM9_cfos = RAM9_bystain_NORM[7]

# adding c-fos high/low labels to single c-fos dataframes
RAM8_cfos_highlow = []
for cfosint in list(RAM8_cfos['Mean-Background_NORM']):
    if cfosint < RAM8_cfos_median:
        RAM8_cfos_highlow.append('c-Fos low')
    elif cfosint >= RAM8_cfos_median:
        RAM8_cfos_highlow.append('c-Fos high')
    
RAM8_cfos['c-Fos Intensity Bin'] = RAM8_cfos_highlow

# repeat for RAM9
RAM9_cfos_highlow = []
for cfosint in list(RAM9_cfos['Mean-Background_NORM']):
    if cfosint < RAM9_cfos_median:
        RAM9_cfos_highlow.append('c-Fos low')
    elif cfosint >= RAM9_cfos_median:
        RAM9_cfos_highlow.append('c-Fos high')
        
RAM9_cfos['c-Fos Intensity Bin'] = RAM9_cfos_highlow
        
RAM89_cfos = pd.concat([RAM8_cfos, RAM9_cfos])
RAM89_cfos
    

,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,Integrated_Density,Max,Mean,Mean-Background,Min,Round,Solidity,Stain,StdDev,Subject,XM,YM,directory,roiName,Rat_n,Coord,Sex,Dox,React,Mean-Background_NORM,c-Fos Intensity Bin
0,1,1.092,_5_3_2021_at_8-34,43.489,56.949,1.0,0.873,RAM008,RAM-8_rat10C_3.4_PL_A,3663.566,217.394,141.189,84.241,59.153999999999996,0.811,0.838,single_c-Fos,43.536,10.0,345.615,79.261,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00002,RAM-8_rat10C,"(345.615, 79.26100000000001)",F,DOX_ON,FR1,0.908207,c-Fos high
1,2,1.000,_5_3_2021_at_8-34,64.533,56.949,2.0,0.968,RAM008,RAM-8_rat10C_3.4_PL_A,5381.915,270.618,140.347,83.398,59.455,0.948,0.911,single_c-Fos,54.939,10.0,57.237,77.429,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00010,RAM-8_rat10C,"(57.236999999999995, 77.429)",F,DOX_ON,FR1,0.899118,c-Fos high
2,3,1.169,_5_3_2021_at_8-34,66.637,56.949,3.0,0.996,RAM008,RAM-8_rat10C_3.4_PL_A,13203.354,478.177,255.088,198.139,78.491,0.846,0.969,single_c-Fos,89.950,10.0,13.054,117.141,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00007,RAM-8_rat10C,"(13.054, 117.141)",F,DOX_ON,FR1,2.136147,c-Fos high
3,4,1.302,_5_3_2021_at_8-34,25.252,56.949,4.0,0.562,RAM008,RAM-8_rat10C_3.4_PL_A,886.135,169.740,92.041,35.092,57.24,0.432,0.545,single_c-Fos,26.349,10.0,368.777,231.472,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00003,RAM-8_rat10C,"(368.777, 231.472)",F,DOX_ON,FR1,0.378329,c-Fos low
4,5,1.083,_5_3_2021_at_8-34,59.623,56.949,5.0,0.981,RAM008,RAM-8_rat10C_3.4_PL_A,8228.827,374.593,194.964,138.015,62.191,0.913,0.924,single_c-Fos,85.767,10.0,107.768,29.112,/Volumes/T7/RAM008_Images/RAM008_10C_cfos405_e...,005-00009,RAM-8_rat10C,"(107.76799999999999, 29.112)",F,DOX_ON,FR1,1.487947,c-Fos high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17773,69,1.000,_5_18_2022_at_16-56,28.759,145.818,69.0,0.431,RAM-9,RAM-9_rat7B_RAM-9_7B_D,1572.359,298.756,200.492,54.673,146.974,0.423,0.406,single_c-Fos,41.351,7D,315.816,299.324,data/RAM-9_7B_D_1/RAM-9_7B_D_1_{single}_{c-Fos...,0358b-0377,RAM-9_rat7B,"(315.816, 299.324)",F,DOX_ON,FR1,0.594239,c-Fos low
17774,70,1.269,_5_18_2022_at_16-56,22.446,145.818,70.0,0.366,RAM-9,RAM-9_rat7B_RAM-9_7B_D,1036.772,263.663,192.008,46.189,145.86700000000002,0.300,0.372,single_c-Fos,30.431,7D,346.980,278.671,data/RAM-9_7B_D_1/RAM-9_7B_D_1_{single}_{c-Fos...,0333b-0414,RAM-9_rat7B,"(346.98, 278.671)",F,DOX_ON,FR1,0.502027,c-Fos low
17775,71,1.209,_5_18_2022_at_16-56,40.684,145.818,71.0,0.511,RAM-9,RAM-9_rat7B_RAM-9_7B_D,2115.181,288.671,197.809,51.991,146.894,0.417,0.513,single_c-Fos,38.575,7D,312.644,323.879,data/RAM-9_7B_D_1/RAM-9_7B_D_1_{single}_{c-Fos...,0386b-0372,RAM-9_rat7B,"(312.644, 323.879)",F,DOX_ON,FR1,0.565088,c-Fos low
17776,72,1.273,_5_18_2022_at_16-56,38.579,145.818,72.0,0.382,RAM-9,RAM-9_rat7B_RAM-9_7B_D,1211.844,263.028,177.230,31.412,146.92700000000002,0.309,0.387,single_c-Fos,26.169,7D,312.291,340.849,data/RAM-9_7B_D_1/RAM-9_7B_D_1_{single}_{c-Fos...,0406b-0373,RAM-9_rat7B,"(312.291, 340.849)",F,DOX_ON,FR1,0.341416,c-Fos low


In [64]:
df = RAM89_cfos

cfos_high_by_image = []
cfos_low_by_image = []
images = []
for rat in np.unique(df['Rat_n']):
    df_rat = df[df.Rat_n == rat]
    
    for image in np.unique(df_rat['FileName']):
        df_rat_image = df_rat[df_rat.FileName == image]
        cfos_high_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos high']))
        cfos_low_by_image.append(len(df_rat_image[df_rat_image['c-Fos Intensity Bin'] == 'c-Fos low']))
        images.append(image)

RAM89_cfoshighlow_counts = pd.DataFrame()
RAM89_cfoshighlow_counts['Image'] = images
RAM89_cfoshighlow_counts['c-Fos high'] = cfos_high_by_image
RAM89_cfoshighlow_counts['c-Fos low'] = cfos_low_by_image

RAM89_cfoshighlow_counts

,Image,c-Fos high,c-Fos low
0,RAM-8_rat10C_3.4_PL_A,9,13
1,RAM-8_rat10C_3.9_PL_A,11,37
2,RAM-8_rat10C_4.2_PL_A,29,21
3,RAM-8_rat13C_3.1_PL_A,14,23
4,RAM-8_rat13C_3.2_PL_A,40,11
...,...,...,...
65,RAM-9_rat5B_RAM-9_5B_D,84,91
66,RAM-9_rat7B_RAM-9_7B_A,55,11
67,RAM-9_rat7B_RAM-9_7B_B,1,22
68,RAM-9_rat7B_RAM-9_7B_C,1,41


In [65]:
rat_ns = []
for image in RAM89_cfoshighlow_counts['Image']:
    rat_n = '_'.join(image.split('_')[0:2])
    rat_ns.append(rat_n)
    
RAM89_cfoshighlow_counts['Rat_n'] = rat_ns

rats = np.unique(RAM89.Rat_n)

SUM_cfos_high = []
SUM_cfos_low = []
rat_ns = []
for rat in rats:
    
    if rat in set(RAM89_cfoshighlow_counts['Rat_n']):
        df = RAM89_cfoshighlow_counts[RAM89_cfoshighlow_counts['Rat_n'] == rat]
        SUM_cfos_high.append(np.sum(df['c-Fos high']))
        SUM_cfos_low.append(np.sum(df['c-Fos low']))
        rat_ns.append(rat)
        
    elif rat not in set(RAM89_cfoshighlow_counts['Rat_n']):
        SUM_cfos_high.append(0)
        SUM_cfos_low.append(0)
        rat_ns.append(rat)
        
    
RAM89_by_cfos = pd.DataFrame()
RAM89_by_cfos['Rat_n'] = rat_ns
RAM89_by_cfos['Stain'] = 'c-Fos'
RAM89_by_cfos['Sum Raw Cell n, c-Fos high'] = SUM_cfos_high
RAM89_by_cfos['Sum Raw Cell n, c-Fos low'] = SUM_cfos_low

RAM89_by_cfos

,Rat_n,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low"
0,RAM-8_rat10C,c-Fos,49,71
1,RAM-8_rat12C,c-Fos,0,0
2,RAM-8_rat13C,c-Fos,196,58
3,RAM-8_rat15C,c-Fos,163,88
4,RAM-8_rat16C,c-Fos,131,58
5,RAM-8_rat1C,c-Fos,103,82
6,RAM-8_rat3C,c-Fos,191,182
7,RAM-8_rat4C,c-Fos,49,202
8,RAM-8_rat5C,c-Fos,188,292
9,RAM-8_rat7C,c-Fos,14,24


In [68]:
rats = np.unique(RAM89.Rat_n)
sex = []
dox = []
react = []
for rat in rats:
    sex.append(np.unique(RAM89[RAM89.Rat_n == rat]['Sex']).item())
    dox.append(np.unique(RAM89[RAM89.Rat_n == rat]['Dox']).item())
    react.append(np.unique(RAM89[RAM89.Rat_n == rat]['React']).item())
    
d_sex = dict(zip(rats, sex))
d_dox = dict(zip(rats, dox))
d_react = dict(zip(rats, react))

d_imgn = {
    'RAM-8_rat10C' : 3,
    'RAM-8_rat12C' : 5, 
    'RAM-8_rat13C' : 5, 
    'RAM-8_rat15C' : 6,
    'RAM-8_rat16C' : 4, 
    'RAM-8_rat1C' : 6, 
    'RAM-8_rat3C' : 3, 
    'RAM-8_rat4C' : 3,
    'RAM-8_rat5C' : 5, 
    'RAM-8_rat7C' : 1, 
    'RAM-8_rat8C' : 3, 
    'RAM-9_rat11B' : 3,
    'RAM-9_rat12B' : 4, 
    'RAM-9_rat14B' : 4, 
    'RAM-9_rat1B' : 4, 
    'RAM-9_rat2B' : 4,
    'RAM-9_rat3B' : 4, 
    'RAM-9_rat5B' : 4, 
    'RAM-9_rat7B' : 4
}

rats = []
sex = []
dox = []
react = []
img_n = []
mean_cell_n_cfoshigh = []
mean_cell_n_cfoslow = []
for rat in RAM89_by_cfos.Rat_n:
    sex.append(d_sex[rat])
    dox.append(d_dox[rat])
    react.append(d_react[rat])
    img_n.append(d_imgn[rat])    
    sum_n = np.array(RAM89_by_cfos[RAM89_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos high']).item()
    mean_cell_n_cfoshigh.append(sum_n / d_imgn[rat])
    sum_n = np.array(RAM89_by_cfos[RAM89_by_cfos['Rat_n'] == rat]['Sum Raw Cell n, c-Fos low']).item()
    mean_cell_n_cfoslow.append(sum_n / d_imgn[rat])

    rats.append(rat)
    
RAM89_by_cfos = RAM89_by_cfos.drop(columns=['Rat_n'])

RAM89_by_cfos['Total Image n'] = img_n
RAM89_by_cfos['Mean Cell n, c-Fos High'] = mean_cell_n_cfoshigh
RAM89_by_cfos['Mean Cell n, c-Fos Low'] = mean_cell_n_cfoslow
RAM89_by_cfos['Rat_n'] = rats
RAM89_by_cfos['Dox'] = dox
RAM89_by_cfos['Sex'] = sex
RAM89_by_cfos['React'] = react

RAM89_by_cfos = RAM89_by_cfos.sort_values(by=['Dox', 'React', 'Sex','Rat_n'])

RAM89_by_cfos

,Stain,"Sum Raw Cell n, c-Fos high","Sum Raw Cell n, c-Fos low",Total Image n,"Mean Cell n, c-Fos High","Mean Cell n, c-Fos Low",Rat_n,Dox,Sex,React
3,c-Fos,163,88,6,27.166667,14.666667,RAM-8_rat15C,DOX_OFF,F,FR1
11,c-Fos,28,55,3,9.333333,18.333333,RAM-9_rat11B,DOX_OFF,F,FR1
7,c-Fos,49,202,3,16.333333,67.333333,RAM-8_rat4C,DOX_OFF,M,FR1
9,c-Fos,14,24,1,14.000000,24.000000,RAM-8_rat7C,DOX_OFF,M,FR1
10,c-Fos,116,143,3,38.666667,47.666667,RAM-8_rat8C,DOX_OFF,M,FR1
16,c-Fos,214,225,4,53.500000,56.250000,RAM-9_rat3B,DOX_OFF,M,FR1
5,c-Fos,103,82,6,17.166667,13.666667,RAM-8_rat1C,DOX_OFF,M,NR
1,c-Fos,0,0,5,0.000000,0.000000,RAM-8_rat12C,DOX_OFF,F,VR5
2,c-Fos,196,58,5,39.200000,11.600000,RAM-8_rat13C,DOX_OFF,F,VR5
12,c-Fos,264,170,4,66.000000,42.500000,RAM-9_rat12B,DOX_OFF,F,VR5


In [69]:
RAM89_by_cfos.to_csv('RAM-8,9 Single c-Fos Mean Cell ns, c-Fos Median Split.csv')
RAM89_cfos.to_csv('RAM-8,9 Single c-Fos Median Split RAW.csv')